As the yfinance package is a powerful tool to combine the technology edge of data analysis and trading finance.
In this project, the main objective is the comparasion between DCF( Discounted Cash Flow) calculation on the stock price and the ML approach of using historical data to predict stock price 

1. Step 1 : Get the DCF result from yfinance
2. Step 2 : Data Prep for ML 
3. Step 3: Building the ML model using Random forest method(X:historical data(trailing PE,open,closed,trading volume) and LSTM approach
4. Step 4: Compare 2 models results to each other and DCL approach from the yfinance data (bank_of_america.info["forwardPE"])

In [2]:
# importing modules 
import pandas as pd
from datetime import datetime
import yfinance as yf
import scipy.stats
import statistics
from scipy.stats import pearsonr, spearmanr, kendalltau
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import seaborn as sns
import pandas_datareader as pdr
import datetime

Data Prep

In [50]:
# Using the yfinance lib 
goldman = yf.Ticker("GS")
jp = yf.Ticker("JPM")
ms = yf.Ticker("MS")

goldman_eps = goldman.income_stmt.T


In [52]:
goldman_eps["Diluted EPS"]

2023-12-31        22.87
2022-12-31        30.06
2021-12-31    59.450001
2020-12-31        24.74
Name: Diluted EPS, dtype: object

## DCF Method Calculation 

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-01-13 00:00:00-05:00,216.283504,217.761139,215.009372,216.964813,3359200,0.0,0.0
2020-01-14 00:00:00-05:00,217.062143,219.893540,216.425077,217.362976,4302800,0.0,0.0
2020-01-15 00:00:00-05:00,214.124560,220.760652,211.611697,216.964813,5411200,0.0,0.0
2020-01-16 00:00:00-05:00,218.557460,221.043794,217.867308,220.955307,3968400,0.0,0.0
2020-01-17 00:00:00-05:00,221.406577,221.610093,219.433449,220.725281,3108400,0.0,0.0
...,...,...,...,...,...,...,...
2025-01-03 00:00:00-05:00,581.000000,582.140015,571.729980,580.130005,1422600,0.0,0.0
2025-01-06 00:00:00-05:00,584.979980,593.650024,579.820007,583.390015,2257000,0.0,0.0
2025-01-07 00:00:00-05:00,587.109985,588.280029,568.400024,580.119995,2021900,0.0,0.0


Bellow is the functions to pull data from yahoo finance from list of selected stocks and put them into 1 DF

In [45]:
def get(stocks, startdate, enddate):
    def data(ticker):
        # Fetch historical data for the ticker
        return yf.Ticker(ticker).history(start=startdate, end=enddate)
    
    # Map the data fetching function to all stocks and concatenate the results
    datas = map(data, stocks)
    return pd.concat(datas, keys=stocks, names=['Ticker', 'Date'])

# Define stock tickers and date range
stocks = ["GS", "MS", "JPM"]
start_date = datetime.datetime(2015, 1, 1)
end_date = datetime.datetime(2019, 1, 1)

# Fetch all data
all_data = get(stocks, start_date, end_date)

# Display the data
all_data.head()

Open        High         Low  \
Ticker Date                                                            
GS     2015-01-02 00:00:00-05:00  160.295621  160.648545  158.161624   
       2015-01-05 00:00:00-05:00  158.457085  159.261433  153.877210   
       2015-01-06 00:00:00-05:00  154.550239  154.845715  150.963483   
       2015-01-07 00:00:00-05:00  153.360135  154.295808  152.473706   
       2015-01-08 00:00:00-05:00  155.412042  156.618569  155.329960   

                                       Close   Volume  Dividends  Stock Splits  
Ticker Date                                                                     
GS     2015-01-02 00:00:00-05:00  159.565140  1877700        0.0           0.0  
       2015-01-05 00:00:00-05:00  154.583069  3413200        0.0           0.0  
       2015-01-06 00:00:00-05:00  151.455948  3429200        0.0           0.0  
       2015-01-07 00:00:00-05:00  153.713058  1896800        0.0           0.0  
       2015-01-08 00:00:00-05:00  156.167145  1962500        0.0           0.0

In [64]:
all_data.iloc[1,2]


KeyError: "None of [Index([1, 2], dtype='int64')] are in the [columns]"

AttributeError: 'Ticker' object has no attribute 'T'

In [151]:
a["Stock Splits"].value_counts()

Stock Splits
0.0    252
Name: count, dtype: int64

In [161]:
bank_of_america.info["trailingPE"]

16.355072

In [162]:
bank_of_america.info["forwardPE"]

12.176239

In [95]:
# Get selected stock current price
citi_price = citi.history(period ='1d')["Close"]
star

The current price 71.65


In [22]:
inc_citi = citi.income_stmt
inc_state = state.income_stmt
inc_bny = bny.income_stmt
inc_ubs = ubs.income_stmt

citi_eps = inc_citi.loc["Diluted EPS"]
state_eps = inc_state.loc["Diluted EPS"]
bny_eps = inc_bny.loc["Diluted EPS"]
ubs_eps = inc_ubs.loc["Diluted EPS"]

citi_v = list(citi_eps)
state_v = list(state_eps)
bny_v = list(bny_eps)
ubs_v = list(ubs_eps)

In [25]:
# Print the EPS of citi bank
ubs_eps

2023-12-31    8.45
2022-12-31    2.25
2021-12-31    2.06
2020-12-31    1.79
Name: Diluted EPS, dtype: object

In [6]:
#  function for removing the last element in the EPS list 
def rem_last(list):
    list.pop()

In [7]:
rem_last(citi_v)
rem_last(state_v)
rem_last(bny_v)
rem_last(ubs_v)

In [8]:
Years= [2023,2022,2021,2020]

In [9]:
eps_dict = {"Years": Years,
    "Citi Group" :citi_v,
            "State Street":state_v,
            "BNY Mellon": bny_v,
            "UBS": ubs_v}

In [10]:
print(eps_dict)

{'Years': [2023, 2022, 2021, 2020], 'Citi Group': [4.04, 7.0, 10.14, 4.87], 'State Street': [5.58, 7.19, 7.19, 6.32], 'BNY Mellon': [4.0, 2.9, 4.14, 3.83], 'UBS': [8.45, 2.25, 2.06, 1.79]}


In [11]:
df = pd.DataFrame(eps_dict)

In [12]:
print(df)

   Years  Citi Group  State Street  BNY Mellon   UBS
0   2023        4.04          5.58        4.00  8.45
1   2022        7.00          7.19        2.90  2.25
2   2021       10.14          7.19        4.14  2.06
3   2020        4.87          6.32        3.83  1.79


In [41]:
# Using income statement elements such as : Tax Provision, Selling And Marketing; General And Administrative Espense 
# to identify the correlation 

# Tax Provision, Selling And Marketing; General And Administrative Espense factors as Correlation Coefficient with EPS from the Income Statement

## Citi Group 

In [13]:
# Extracting chosen factors 
citi_tax = inc_citi.loc["Tax Provision"]
citi_selling_marketing = inc_citi.loc["Selling And Marketing Expense"]
citi_gen_admin = inc_citi.loc["General And Administrative Expense"]

citi_dict ={"Tax Provision" : citi_tax,
            "Selling And Marketing Expense": citi_selling_marketing,
            "General And Administrative Expense": citi_gen_admin
}
df_citi = pd.DataFrame(citi_dict)


In [14]:
df_citi

,Tax Provision,Selling And Marketing Expense,General And Administrative Expense
2023-12-31,3528000000.0,1393000000.0,29319000000.0
2022-12-31,3642000000.0,1556000000.0,26749000000.0
2021-12-31,5451000000.0,1490000000.0,25250000000.0
2020-12-31,2525000000.0,1217000000.0,22327000000.0
2019-12-31,NaN,NaN,NaN


In [15]:
# Drop the last row
df_citi.drop(df_citi.tail(1).index,inplace=True)

In [16]:
x = df["Citi Group"]
y = df_citi["General And Administrative Expense"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: -0.1899441505103251
Spearman correlation coefficient: -0.39999999999999997
Kendall correlation coefficient: -0.3333333333333334


In [17]:
x = df["Citi Group"]
y = df_citi["Tax Provision"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: 0.873055098363501
Spearman correlation coefficient: 0.7999999999999999
Kendall correlation coefficient: 0.6666666666666669


When we look at the Pearson correlation coefficient, we retrieved that the value 0.598 with the identification that there is moderate correlation between Earning per share and Selling and Marketing Expense. We can see that, there is acutually weak negativecorrelation between Earning Per Share and General And Administrative Expense. However, when we look at the Tax Provision part, we can see that there is strong correlation between the value of Tax Provision and Earning Per Share 

## State Street

In [18]:

state_tax = inc_state.loc["Tax Provision"]
state_selling_marketing = inc_state.loc["Selling And Marketing Expense"]
state_gen_admin = inc_state.loc["General And Administrative Expense"]

state_dict ={"Tax Provision" : state_tax,
            "Selling And Marketing Expense": state_selling_marketing,
            "General And Administrative Expense": state_gen_admin
}
df_state = pd.DataFrame(state_dict)

In [19]:
df_state

,Tax Provision,Selling And Marketing Expense,General And Administrative Expense
2023-12-31,372000000.0,142000000.0,4562000000.0
2022-12-31,553000000.0,99000000.0,4378000000.0
2021-12-31,478000000.0,73000000.0,4586000000.0
2020-12-31,479000000.0,77000000.0,4368000000.0
2019-12-31,NaN,NaN,NaN


In [56]:
# Drop the last row
df_state.drop(df_state.tail(1).index,inplace=True)

In [57]:
x = df["State Street"]
y = df_state["Selling And Marketing Expense"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: -0.7212124593500673
Spearman correlation coefficient: -0.632455532033676
Kendall correlation coefficient: -0.5477225575051662


In [131]:
x = df["State Street"]
y = df_state["General And Administrative Expense"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: -0.18663607623338274
Spearman correlation coefficient: 0.21081851067789195
Kendall correlation coefficient: 0.18257418583505539


In [132]:
x = df["State Street"]
y = df_state["Tax Provision"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: 0.8709202268580564
Spearman correlation coefficient: 0.632455532033676
Kendall correlation coefficient: 0.5477225575051662


When we look at the Pearson correlation coefficient between Earning per share and Selling and Marketing Expense. Similarly, we can see that, there is acutually no correlation between Earning Per Share and General And Administrative Expense. However, when we look at the Tax Provision part, we can see that there is strong correlation between the value of Tax Provision and Earning Per Share. 

In [96]:
df_state

,Tax Provision,Selling And Marketing Expense,General And Administrative Expense
2023-12-31,372000000.0,142000000.0,4562000000.0
2022-12-31,553000000.0,99000000.0,4378000000.0
2021-12-31,478000000.0,73000000.0,4586000000.0
2020-12-31,479000000.0,77000000.0,4368000000.0


## BNY Mellon

In [59]:
bny_tax = inc_bny.loc["Tax Provision"]
bny_gen_admin = inc_bny.loc["General And Administrative Expense"]

bny_dict ={"Tax Provision" : bny_tax,
            "General And Administrative Expense": bny_gen_admin}
df_bny = pd.DataFrame(bny_dict)

In [60]:
df_bny

,Tax Provision,General And Administrative Expense
2023-12-31,800000000.0,6828000000.0
2022-12-31,768000000.0,6585000000.0
2021-12-31,877000000.0,6337000000.0
2020-12-31,842000000.0,5966000000.0
2019-12-31,NaN,NaN


In [63]:
# Drop the last row
df_bny.drop(df_bny.tail(1).index,inplace=True)

In [64]:
df_bny

,Tax Provision,General And Administrative Expense
2023-12-31,800000000.0,6828000000.0
2022-12-31,768000000.0,6585000000.0
2021-12-31,877000000.0,6337000000.0
2020-12-31,842000000.0,5966000000.0


In [136]:
x = df["BNY Mellon"]
y = df_bny["Tax Provision"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: 0.8555371012512922
Spearman correlation coefficient: 0.7999999999999999
Kendall correlation coefficient: 0.6666666666666669


In [138]:
x = df["BNY Mellon"]
y = df_bny["General And Administrative Expense"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: -0.26383640688331556
Spearman correlation coefficient: 0.0
Kendall correlation coefficient: 0.0


## UBS

In [97]:
ubs_tax = inc_ubs.loc["Tax Provision"]
ubs_selling_marketing = inc_ubs.loc["Selling And Marketing Expense"]
ubs_gen_admin = inc_ubs.loc["General And Administrative Expense"]

ubs_dict ={"Tax Provision" : ubs_tax,
            "Selling And Marketing Expense": ubs_selling_marketing,
            "General And Administrative Expense": ubs_gen_admin
}
df_ubs = pd.DataFrame(ubs_dict)

In [98]:
df_ubs

,Tax Provision,Selling And Marketing Expense,General And Administrative Expense
2023-12-31,873000000.0,408000000.0,27189000000.0
2022-12-31,1942000000.0,265000000.0,21059000000.0
2021-12-31,1998000000.0,242000000.0,21612000000.0
2020-12-31,1583000000.0,217000000.0,20378000000.0
2019-12-31,NaN,NaN,NaN


In [99]:
df_ubs.drop(df_ubs.tail(1).index,inplace=True)

In [139]:
x = df["UBS"]
y = df_ubs["General And Administrative Expense"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: 0.9911742372284684
Spearman correlation coefficient: 0.7999999999999999
Kendall correlation coefficient: 0.6666666666666669


In [140]:
x = df["UBS"]
y = df_ubs["Selling And Marketing Expense"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: 0.9851611178897732
Spearman correlation coefficient: 1.0
Kendall correlation coefficient: 1.0


In [141]:
x = df["UBS"]
y = df_ubs["Tax Provision"]
pearson_coef, _ = pearsonr(x, y)
print("Pearson correlation coefficient:", pearson_coef)

spearman_coef, _ = spearmanr(x, y)
print("Spearman correlation coefficient:", spearman_coef)

kendall_coef, _ = kendalltau(x, y)
print("Kendall correlation coefficient:", kendall_coef)

Pearson correlation coefficient: -0.9154096199008013
Spearman correlation coefficient: -0.39999999999999997
Kendall correlation coefficient: -0.3333333333333334


Coefficient of Determination 
https://www.investopedia.com/terms/c/coefficient-of-determination.asp



This coefficient generally answers the following question: If a stock is listed on an index and experiences price movements, what percentage of its price movement is attributed to the index's price movement?
    


In [40]:
# Setup the start and end time stamp for the time series
end = datetime.now()
start = datetime(end.year - 4,end.month,end.day)

In [128]:
# Using global methods to 
for stock in fin_list:
    globals()[stock] = yf.download(stock, start, end)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [91]:
company_list = [STT, C, BK, UBS]
company_name = ["State Street", "Citi Bank", "BNY MELLON ", "UBS"]

In [190]:
for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name


In [122]:
df = pd.concat(company_list, axis=0)
x= STT["Open"][].mean()
y = C["Open"].mean()

In [189]:
print(x)

76.3961210856362
